In [48]:
import pandas as pd

df_omit = pd.read_csv('./data/아파트(매매)__실거래가_2013.csv',encoding='euc-kr')
df_omit.head(3)

,시군구,번지,본번,부번,단지명,전용면적(㎡),계약년월,계약일,거래금액(만원),층,건축년도,도로명,해제사유발생일,거래유형,중개사소재지
0,서울특별시 강남구 개포동,655-2,655,2,개포2차현대아파트(220),77.75,201309,8,"57,000",2,1988,언주로 103,NaN,-,-
1,서울특별시 강남구 개포동,655-2,655,2,개포2차현대아파트(220),77.75,201312,16,"57,000",2,1988,언주로 103,NaN,-,-
2,서울특별시 강남구 개포동,658-1,658,1,개포6차우성아파트1동~8동,67.28,201302,11,"55,000",5,1987,언주로 3,NaN,-,-


In [49]:
df_drop_table = df_omit.drop(['해제사유발생일','중개사소재지','거래유형'],axis=1)

In [50]:
df_drop_table.isna().sum()

시군구         0
번지          0
본번          0
부번          0
단지명         0
전용면적(㎡)     0
계약년월        0
계약일         0
거래금액(만원)    0
층           0
건축년도        0
도로명         0
dtype: int64

In [51]:
main = df_drop_table['본번'].astype(str).tolist()
sub = df_drop_table['부번'].astype(str).tolist()

temp = []
for i,j in zip(main,sub):
    if j!='0':
        temp.append('-'.join([i,j]))
    else:
        temp.append(i)

df_new_num_addr = df_drop_table.copy()
df_new_num_addr.loc[:,'번지'] = temp
df_new_num_addr.head(3)

,시군구,번지,본번,부번,단지명,전용면적(㎡),계약년월,계약일,거래금액(만원),층,건축년도,도로명
0,서울특별시 강남구 개포동,655-2,655,2,개포2차현대아파트(220),77.75,201309,8,"57,000",2,1988,언주로 103
1,서울특별시 강남구 개포동,655-2,655,2,개포2차현대아파트(220),77.75,201312,16,"57,000",2,1988,언주로 103
2,서울특별시 강남구 개포동,658-1,658,1,개포6차우성아파트1동~8동,67.28,201302,11,"55,000",5,1987,언주로 3


In [52]:
import env
import googlemaps
maps = googlemaps.Client(key=env.KEY)  # my key값 입력

df_lat_lng = df_new_num_addr.copy()
address = pd.Series(name='주소',data = list(map(lambda x: ' '.join(x),df_lat_lng[['시군구','도로명']].to_numpy())))
lat,lng = [],[]

for i in address.unique():
    try:
        location = maps.geocode(i)[0].get('geometry')
        lat.append(location['location']['lat'])
        lng.append(location['location']['lng'])
    except Exception as e:
        print(e)
        print('ERROR : ',i)

In [ ]:
pos = pd.DataFrame({'위도':lat, '경도':lng,'주소':address.unique().tolist()})
pos.to_csv('./processed/pos_2013.csv')
add_address = pd.concat([df_lat_lng.copy(),address],axis=1)
final = add_address.merge(pos,on='주소',how='left',suffixes=('_',''))

# check data
final.to_csv('./processed/preprocessed_2013.csv')